In [ ]:
import torch
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.visualizer import Visualizer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
import os
import random
import cv2
import matplotlib.pyplot as plt

# Ensure the output is displayed inline
%matplotlib inline

In [ ]:
# Replace with your paths to the COCO dataset
coco_train_images = "datasets/coco/train2017"
coco_train_annotations = "datasets/coco/annotations/instances_train2017.json"

coco_val_images = "datasets/coco/val2017"
coco_val_annotations = "datasets/coco/annotations/instances_val2017.json"

# Optional: test dataset
coco_test_images = "datasets/coco/test2017"
coco_test_annotations = "datasets/coco/annotations/instances_test2017.json"

# Register datasets
register_coco_instances("coco_train", {}, coco_train_annotations, coco_train_images)
register_coco_instances("coco_val", {}, coco_val_annotations, coco_val_images)

# Optional: Test dataset registration
register_coco_instances("coco_test", {}, coco_test_annotations, coco_test_images)

# Fetch metadata for visualization
train_metadata = MetadataCatalog.get("coco_train")
val_metadata = MetadataCatalog.get("coco_val")
test_metadata = MetadataCatalog.get("coco_test")


In [ ]:
cfg = get_cfg()

# Load the Cascade Mask R-CNN config file
cfg.merge_from_file("detectron2/configs/COCO-InstanceSegmentation/cascade_mask_rcnn_R_50_FPN_3x.yaml")

# Set dataset paths
cfg.DATASETS.TRAIN = ("coco_train",)
cfg.DATASETS.TEST = ("coco_val",)  # For validation

# Adjust workers and batch sizes
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2  # Batch size per iteration

# Set the base learning rate and iterations
cfg.SOLVER.BASE_LR = 0.0025
cfg.SOLVER.MAX_ITER = 30000  # Set based on your needs

# Specify model architecture parameters
cfg.MODEL.WEIGHTS = "detectron2://ImageNetPretrained/MSRA/R-50.pkl"  # Pretrained weights
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512  # RoI head batch size
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 80  # COCO has 80 instance categories

# Enable instance segmentation (masks)
cfg.MODEL.MASK_ON = True

# Set output directory to store model checkpoints and logs
cfg.OUTPUT_DIR = "./output_cascade_mask_rcnn"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)


In [ ]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# COCO evaluator to measure performance
evaluator = COCOEvaluator("coco_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "coco_val")

# Perform inference on the validation dataset
inference_on_dataset(trainer.model, val_loader, evaluator)


In [ ]:
# Visualize random validation samples with predictions
dataset_dicts = DatasetCatalog.get("coco_val")
for d in random.sample(dataset_dicts, 3):  # Visualize 3 random samples
    img = cv2.imread(d["file_name"])
    
    # Use Visualizer to draw the predictions
    visualizer = Visualizer(img[:, :, ::-1], metadata=val_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    
    # Display the image using matplotlib
    plt.figure(figsize=(10, 10))
    plt.imshow(vis.get_image()[:, :, ::-1])
    plt.axis("off")
    plt.show()


In [ ]:
# Save the model's last checkpoint
trainer.checkpointer.save("cascade_mask_rcnn_final")

# Evaluate on the test dataset (optional)
cfg.DATASETS.TEST = ("coco_test",)  # Use the test set
test_loader = build_detection_test_loader(cfg, "coco_test")
evaluator = COCOEvaluator("coco_test", cfg, False, output_dir="./output/")
inference_on_dataset(trainer.model, test_loader, evaluator)

In [ ]:
# Step 1: Install Required Libraries (Run this in a Jupyter Notebook cell)
# !pip install 'git+https://github.com/facebookresearch/detectron2.git'

# Step 2: Import Necessary Libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Step 3: Set Up the Configuration
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set the threshold for this model

# Step 4: Initialize the Predictor
predictor = DefaultPredictor(cfg)

# Step 5: Load an Image
image_path = "path/to/your/image.jpg"  # Update this path to your image file
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB

# Step 6: Run Inference
outputs = predictor(image)

# Step 7: Visualize the Predictions
v = Visualizer(image, MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

# Display the results
plt.figure(figsize=(10, 10))
plt.imshow(v.get_image())
plt.axis('off')  # Turn off axis numbers and ticks
plt.show()
